In [8]:
# auto reload modules
%load_ext autoreload
%autoreload 2

In [9]:
import numpy as np
import json

In [10]:
#import PES_models as pmodel, PES_models_bonding_features as bond_pmodel
from PES_models_bonding_features import *
from PES_data_processor import *

### save Hn as numpy array

In [17]:
f = open("data/h4/H4_Hinde_QZ.txt")
data = json.load(f)
H4 = np.array(data['data'])
np.savetxt("data/h4/h4_QZ_data.txt", H4)

f = open("data/h4/H4_Hinde_TZ.txt")
data = json.load(f)
H4 = np.array(data['data'])
np.savetxt("data/h4/h4_TZ_data.txt", H4)

TZ


In [2]:
f = open("data/h5/H5_Tablero.txt")
data = json.load(f)
H5 = np.array(data['data'])
np.savetxt("data/h5/H5_data.txt", H5)

### check X

In [50]:
Hn = np.loadtxt("data/h4/h4_TZ_data.txt")
R = Hn[:, :-1]
idx = 100
print(R[idx])
V = Hn[:, -1]
X = np.load("data/h4/h4_TZ_coord.npy")
print(X.shape)
print(X[idx])
print(np.linalg.norm(X[idx][2] - X[idx][3]))

[1.1        4.50249931 4.67038542 4.67038542 4.50249931 1.4       ]
(855, 4, 3)
[[-2.96666523e-08  5.50000000e-01 -2.25000000e+00]
 [ 1.74017625e-08 -5.50000000e-01 -2.25000000e+00]
 [ 1.23587181e-08  7.00000000e-01  2.25000000e+00]
 [-2.46236079e-08 -7.00000000e-01  2.25000000e+00]]
1.4


### split data directly for every functional other than our PES 

In [57]:
Hn = np.load("data/h3/h3_data.npy")
cv_idx = np.load("data/h3/crossval_indices_1.npy",allow_pickle=True)
train = Hn[cv_idx[0]]; test = Hn[cv_idx[1]]
R_train = train[:, :-1]
V_train = train[:, -1]
R_test = test[:, :-1]
V_test = test[:, -1]

In [61]:
# save to file:
# ab initio (training) data:
with open('data/h3/abinitio_data.txt', "w") as f:
    for i in range(R_train.shape[0]):
        string = ""
        for j in range(R_train.shape[1]):
            string += str(R_train[i][j])+"\t"
        f.write(string+str(V_train[i])+"\t"+str(1.000)+"\n")

with open('data/h3/test_data.txt', "w") as f:
    for i in range(R_test.shape[0]):
        string = ""
        for j in range(R_test.shape[1]):
            string += str(R_test[i][j])+"\t"
        f.write(string+str(V_test[i])+"\t"+str(1.000)+"\n")

### check preds and RMSE (PES)

In [68]:
# load data:
data = np.loadtxt("data/h4/h4_QZ_data.txt") # .npy for H3, .txt for anything else
X = np.load("data/h4/h4_QZ_coord.npy")
cv_idx = np.load("data/h4/indices_QZ/crossval_indices_1.npy",allow_pickle=True)
train = data[cv_idx[0]]; test = data[cv_idx[1]]
X_test = X[cv_idx[1]]
R_test = test[:, :-1]
V_test = test[:, -1]


# fixed parameters:
num_basis = 59; max_deg = 5; num_atom = 4; e = 3; g = 6;

# indexer matrix:
indexer = atom_indexer(num_atom)

C0 = np.loadtxt("params/h4/c_params_070422_full_fold1_h4_QZ_8e-3_3e-1.out") # optimized parameter
start = time.time()
V_pred = f_pot_bond_wrapper_trpp(C0, num_basis, R_test, X_test, indexer, num_atom, max_deg, e, g)
print(time.time()-start, "s")
# RMSE:
rmse = RMSE(V_pred, V_test)
print('rmse', rmse)
for i, pred in enumerate(V_pred):
    print(V_pred[i], V_test[i])

0.002501964569091797 s
rmse 1.7063738457541575
-3.677772231631733 -2.2950488732
-3.736255992450252 -2.2956569843
-3.7332477731242895 -2.2958077913
-3.782629212872788 -2.2959068964
-3.821560589835787 -2.3200165063
-3.788033634682879 -2.3041433642
-3.669046473365214 -2.3060384857
-3.784522633473792 -2.3074966066
-3.7842259835277403 -2.3074519332
-3.8230448563690356 -2.3074330956
-3.8251148806681927 -2.3076278647
-3.7565360807964443 -2.3422719905
-3.842149310458894 -2.3435281001
-3.7339868042116935 -2.3434812509
-3.856323247511303 -2.3438157793
-3.8574982911180253 -2.3439242543
-3.6953977195302907 -2.3296095361
-3.809018613859112 -2.3314334087
-4.21076440133847 -2.3139475206
-3.8499797276161125 -2.3184492888
-3.845760430933966 -2.2970418156
-3.8315723021697483 -2.3211810082
-3.8799390218494887 -2.3213480983
-3.8678618610343056 -2.3453851809
-4.194524328152556 -2.3174815599
-3.8293038418620027 -2.2976564201
-3.8913719543555616 -2.2977859255
-3.7337074431252866 -2.3212303273
-3.888450304924